<a href="https://colab.research.google.com/github/c00jsw00/reactivedock/blob/main/reactive_docking_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## reactive docking with autodock and database by Yeng-Tseng wang (c00jsw00@gmail.com)
This Google Colab notebook

## Step 1. Install OpenBabel, RDKit and Py3Dmol tools --by wang YT

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ambertools -y
!conda install cudatoolkit=12.0

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [1]:
!mamba install rdkit
!mamba install openbabel
#!mamba install cudatoolkit=12.1
!mamba install openmmforcefields -c conda-forge -y
!mamba install openmm
!pip install prolif==1.1.0
#!mamba install -c conda-forge ambertools -y
!mamba install -c conda-forge parmed -y
!pip install --upgrade MDAnalysis
!pip install py3Dmol
!pip install meeko
!pip install prody


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

warning  libmamba Could not parse state file: Could not load cache state: [json.exception

Installing the rdkit patkage

In [2]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(rdBase.rdkitVersion)

2023.09.3


## Step 2. Install autodock-gpu

In [ ]:
!wget https://github.com/c00jsw00/reactivedock/blob/main/autodock_gpu_256wi
!chmod 755 autodock_gpu_256wi

--2023-12-08 08:54:30--  https://github.com/c00jsw00/reactivedock/blob/main/autodock_gpu_256wi
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4696 (4.6K) [text/plain]
Saving to: ‘autodock_gpu_256wi’

autodock_gpu_256wi  100%[===================>]   4.59K  --.-KB/s    in 0s      

2023-12-08 08:54:30 (54.8 MB/s) - ‘autodock_gpu_256wi’ saved [4696/4696]



## Step3: install waterkit and autogrid4

In [5]:
!git clone https://github.com/forlilab/waterkit
!pip install -e waterkit/
!git clone https://github.com/c00jsw00/reactivedock.git
!chmod 755 reactivedock/autogrid4
!chmod 755 reactivedock/autodock_gpu_256wi

## Step4: upload and repair receptor file

In [6]:
from google.colab import files
uploaded = files.upload()
!wk_prepare_receptor.py -i rec.pdb -o rec_prepared --pdb --amber_pdbqt

Saving rec.pdb to rec.pdb
INFO:WaterKit receptor preparation:Removed all hydrogen atoms
INFO:WaterKit receptor preparation:Removed all water molecules
INFO:WaterKit receptor preparation:Histidine protonation states were automatically set to: HIE - 41, HIE - 64, HIE - 80, HIE - 163, HIE - 164, HIE - 172, HIE - 246
INFO:WaterKit receptor preparation:Removed all alternatives residue sidechains


In [7]:
!mk_prepare_receptor.py\
    --pdb rec_prepared.pdb\
    -o receptor.pdbqt\
    --flexres "A:HIE:41" --flexres "A:SER:144" --flexres "A:HIE:164"\
    --reactive_flexres "A:CYS:145"\
    --box_center_on_reactive_res\
    --box_size 40 40 40  # x y z (angstroms)



Flexible residues:
chain resname resnum is_reactive reactive_atom
    A     SER    144       False              
    A     HIE    164       False              
    A     HIE     41       False              
    A     CYS    145        True            SG


For reactive docking, pass the configuration file to AutoDock-GPU:
    autodock_gpu -C 1 --import_dpf receptor.reactive_config --flexres receptor_flex.pdbqt -L <ligand_filename>


Files written:
       receptor_flex.pdbqt <-- flexible receptor input file
      receptor_rigid.pdbqt <-- static (i.e., rigid) receptor input file
boron-silicon-atom_par.dat <-- atomic parameters for B and Si (for autogrid)
        receptor_rigid.gpf <-- autogrid input file
    receptor_rigid.gpf.pdb <-- PDB file to visualize the grid box
  receptor.reactive_config <-- reactive parameters for AutoDock-GPU


### smile

In [ ]:
!rm -rf *.mol2
!rm -rf lig.smile
!rm -rf *.sdf

#import py3Dmol
#from openbabel import pybel
#from rdkit import Chem
#from rdkit.Chem import AllChem
#from rdkit.Chem.Draw import MolToImage

#ref: https://leonis.cc/sui-sui-nian/2023-02-06-rdkit-modify-substructure.html
#ref: https://www.twblogs.net/a/5ee013eb74efa30adcc74198
#ref: https://chem-workflows.com/content/VirtualScreening.html
#ref: https://xinhaoli74.github.io/posts/2020/04/RDKit-Cheatsheet/
#ref: https://www.bigbrosci.com/2020/09/19/A20/
#ref: https://zhuanlan.zhihu.com/p/587181480
#ref: https://colab.research.google.com/github/MobleyLab/drug-computing/blob/master/uci-pharmsci/lectures/3D_structure_shape/3D_Structure_Shape.ipynb#scrollTo=H0IGjxJMIfCC
#Nirmatrelvir
#smiles = "CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C"
#mol = Chem.MolFromSmiles(smiles)
#Warhead_pattern = Chem.MolFromSmarts("[C]=N") #C(=N)SC
#repl = Chem.MolFromSmiles('C(=N)SC')
#....................................
#GC373: CSC(=O)[C@@H](CC1CCNC1=O)NC(=O)[C@@H](CC(C)C)NC(=O)OCc2ccccc2
#GC376: CSC(=O)[C@@H](CC1CCNC1=O)NC(=O)[C@@H](CC(C)C)NC(=O)OCc2ccccc2
#compound4: CSC(=O)[C@H](C[C@@H]1CCNC1=O)NC(=O)C(NC(=O)OCc2cccc(F)c2)C3CC3
#smiles = "CSC(=O)[C@H](C[C@@H]1CCNC1=O)NC(=O)C(NC(=O)OCc2cccc(F)c2)C3CC3"
#mol = Chem.MolFromSmiles(smiles)
#Warhead_pattern = Chem.MolFromSmarts("C=O") #C(=O)SC
#repl = Chem.MolFromSmiles('C(=O)SC')
#rms = AllChem.ReplaceSubstructs(mol, Warhead_pattern, repl)
#mh = Chem.AddHs(rms[0])
#mol = rms[0]
#mol #drawer 2D image
#smi = Chem.MolToSmiles(mol) # convert mol to smile file format
#fo = open("lig.smile", "w")
#fo.write(smi)
#fo.close()

#mol2
#out=pybel.Outputfile(filename='Nirmatrelvir.mol2',format='mol2',overwrite=True)
#mol2 = pybel.readstring("smi", smi)
#mol2.make3D(forcefield='mmff94', steps=100)
#out.write(mol2)
#out.close()
##
#!obabel Nirmatrelvir.mol2 -O Nirmatrelvir.sdf -m

1 molecule converted


In [ ]:
#!rm -rf all.tar
#!rm -rf Nirmatrelvir.pdbqt
#!mk_prepare_ligand.py -i Nirmatrelvir1.sdf --reactive_smarts "C(#N)" --reactive_smarts_idx 1 -o Nirmatrelvir.pdbqt

##Step 5 : running a reactive docking

In [ ]:
#!sudo apt install nvidia-driver-530 -y

In [8]:
!reactivedock/autogrid4 -p  receptor_rigid.gpf
#!./adgpu-v1.5.3_linux_ocl_128wi -C 1 --import_dpf receptor.reactive_config --flexres receptor_flex.pdbqt -L Nirmatrelvir.pdbqt

串流輸出內容已截斷至最後 5000 行。
Atom no. 1158, "C   " at (-0.799, -10.801, 11.335),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1159, "O   " at (-0.726, -9.731, 11.952),  was assigned atom type "OA" (rec_index= 3, atom_type= 3).
Atom no. 1160, "N   " at (-0.693, -10.847, 9.984),  was assigned atom type "N" (rec_index= 0, atom_type= 0).
Atom no. 1161, "CD  " at (-0.402, -12.057, 9.194),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1162, "CG  " at (0.167, -11.491, 7.903),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1163, "CB  " at (-0.554, -10.165, 7.731),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1164, "CA  " at (-0.640, -9.630, 9.168),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1165, "C   " at (-1.857, -8.711, 9.331),  was assigned atom type "C" (rec_index= 7, atom_type= 7).
Atom no. 1166, "O   " at (-2.973, -9.197, 9.339),  was assigned atom type "OA" (rec_index= 3, atom_type= 3).

## Step 6: database

In [9]:
!rm -rf *.csv
from google.colab import files
uploaded = files.upload()# cvs file

Saving database.csv to database.csv


In [10]:
import csv
import py3Dmol
import os
from openbabel import pybel
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import MolToImage
#
with open('database.csv', 'r', newline='') as csvfile:
    #
    csv_reader = csv.reader(csvfile)
    #
    with open('pdbqt.sh', 'w') as file:
        for row in csv_reader:
            parameter1, parameter2, parameter3, parameter4, parameter5, parameter6  = row[0], row[1], row[2], row[3], row[4], row[5]
            #print(f'{parameter1}, {parameter2}, {parameter3}, {parameter4}, {parameter5}')
            #parameter5 = float(row[4])
            name = parameter1+'.sdf'
            smiles = parameter2
            mol = Chem.MolFromSmiles(smiles)
            smi = Chem.MolToSmiles(mol)
            out=pybel.Outputfile(filename=name,format='sdf',overwrite=True)
            sdf = pybel.readstring("smi", smi)
            sdf.make3D(forcefield='mmff94', steps=100)
            out.write(sdf)
            out.close()
            cmd1 ='mk_prepare_ligand.py -i '+parameter1+'.sdf'+' '+'--reactive_smarts '+'"'+parameter3+'"'+' '+'--reactive_smarts_idx '+parameter4+' -o '+parameter1+'.pdbqt'
            file.write(f'{cmd1}\n')

In [11]:
!sh pdbqt.sh
!ls -l

total 150096
-rw-r--r-- 1 root root    4210 Dec  8 14:22 Boceprevir.pdbqt
-rw-r--r-- 1 root root    7665 Dec  8 14:22 Boceprevir.sdf
-rw-r--r-- 1 root root     142 Dec  8 14:20 boron-silicon-atom_par.dat
-rw-r--r-- 1 root root    3189 Dec  8 14:22 calpain_inhibitors_I.pdbqt
-rw-r--r-- 1 root root    5943 Dec  8 14:22 calpain_inhibitors_I.sdf
-rw-r--r-- 1 root root    3935 Dec  8 14:23 Calpain_Inhibitor_XII.pdbqt
-rw-r--r-- 1 root root    6447 Dec  8 14:22 Calpain_Inhibitor_XII.sdf
-rw-r--r-- 1 root root    4839 Dec  8 14:23 CI006628.pdbqt
-rw-r--r-- 1 root root    7871 Dec  8 14:22 CI006628.sdf
-rw-r--r-- 1 root root    4734 Dec  8 14:23 CI006629.pdbqt
-rw-r--r-- 1 root root    7319 Dec  8 14:22 CI006629.sdf
-rw-r--r-- 1 root root    4871 Dec  8 14:23 CI006630.pdbqt
-rw-r--r-- 1 root root    7687 Dec  8 14:22 CI006630.sdf
-rw-r--r-- 1 root root    4871 Dec  8 14:23 CI006632.pdbqt
-rw-r--r-- 1 root root    7411 Dec  8 14:22 CI006632.sdf
-rw-r--r-- 1 root root    4873 Dec  8 14:23 CI0066

In [14]:
!ls -l reactivedock/
!chmod 755 reactivedock/autodock_gpu_128wi
!chmod 755 reactivedock/autodock_gpu_256wi

total 2956
-rw-r--r-- 1 root root  446136 Dec  8 14:19 autodock_gpu_128wi
-rwxr-xr-x 1 root root  446136 Dec  8 14:19 autodock_gpu_256wi
-rwxr-xr-x 1 root root  467936 Dec  8 14:19 autogrid4
-rw-r--r-- 1 root root    4345 Dec  8 14:19 database.csv
-rw-r--r-- 1 root root   11357 Dec  8 14:19 LICENSE
-rw-r--r-- 1 root root  105919 Dec  8 14:19 lipinski.ipynb
-rw-r--r-- 1 root root  296893 Dec  8 14:19 pre_reactive_docking.ipynb
-rw-r--r-- 1 root root  143331 Dec  8 14:19 pre_reactive_docking_V02.ipynb
-rw-r--r-- 1 root root 1088608 Dec  8 14:19 reactive_docking.ipynb
-rw-r--r-- 1 root root      14 Dec  8 14:19 README.md


In [31]:
#libcudart.so.12
#!ln -s -f /usr/local/cuda/lib64/libcudart.so /usr/local/cuda/lib64/libcudart.so.12
#!ls -l /usr/local/cuda/lib64/libcudart*

/usr/local/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:19: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [28]:
%%shell
for f in *.pdbqt;
    do  b=`basename $f .pdbqt`;
    echo Processing ligand $b;
    #./qvina-w --config conf --ligand $f --out $f-out.pdbqt --log $f-out.txt;
    reactivedock/autodock_gpu_128wi -C 1 --import_dpf receptor.reactive_config --flexres receptor_flex.pdbqt -L $f
done

Processing ligand Boceprevir
reactivedock/autodock_gpu_128wi: /usr/local/cuda/lib64/libcudart.so.12: version `libcudart.so.12' not found (required by reactivedock/autodock_gpu_128wi)
Processing ligand calpain_inhibitors_I
reactivedock/autodock_gpu_128wi: /usr/local/cuda/lib64/libcudart.so.12: version `libcudart.so.12' not found (required by reactivedock/autodock_gpu_128wi)
Processing ligand Calpain_Inhibitor_XII
reactivedock/autodock_gpu_128wi: /usr/local/cuda/lib64/libcudart.so.12: version `libcudart.so.12' not found (required by reactivedock/autodock_gpu_128wi)
Processing ligand CI006628
reactivedock/autodock_gpu_128wi: /usr/local/cuda/lib64/libcudart.so.12: version `libcudart.so.12' not found (required by reactivedock/autodock_gpu_128wi)
Processing ligand CI006629
reactivedock/autodock_gpu_128wi: /usr/local/cuda/lib64/libcudart.so.12: version `libcudart.so.12' not found (required by reactivedock/autodock_gpu_128wi)
Processing ligand CI006630
reactivedock/autodock_gpu_128wi: /usr/loc

CalledProcessError: ignored

Analyzing the docking results

In [ ]:
!ls -l

total 161444
-rwxr-xr-x 1 root root  592832 Dec 17  2021 adgpu-v1.5.3_linux_ocl_128wi
-rw-r--r-- 1 root root  293908 Dec  4 16:25 Boceprevir.dlg
-rw-r--r-- 1 root root    4210 Dec  4 16:24 Boceprevir.pdbqt
-rw-r--r-- 1 root root    7665 Dec  4 16:23 Boceprevir.sdf
-rw-r--r-- 1 root root   93764 Dec  4 16:25 Boceprevir.xml
-rw-r--r-- 1 root root     142 Dec  4 16:16 boron-silicon-atom_par.dat
-rw-r--r-- 1 root root  261805 Dec  4 16:25 calpain_inhibitors_I.dlg
-rw-r--r-- 1 root root    3189 Dec  4 16:24 calpain_inhibitors_I.pdbqt
-rw-r--r-- 1 root root    5943 Dec  4 16:23 calpain_inhibitors_I.sdf
-rw-r--r-- 1 root root   79047 Dec  4 16:25 calpain_inhibitors_I.xml
-rw-r--r-- 1 root root  286418 Dec  4 16:26 Calpain_Inhibitor_XII.dlg
-rw-r--r-- 1 root root    3935 Dec  4 16:24 Calpain_Inhibitor_XII.pdbqt
-rw-r--r-- 1 root root    6447 Dec  4 16:23 Calpain_Inhibitor_XII.sdf
-rw-r--r-- 1 root root   92815 Dec  4 16:26 Calpain_Inhibitor_XII.xml
-rw-r--r-- 1 root root  308026 Dec  4 16:27 C

In [ ]:
import glob

def extract_efb_from_dlg(dlg_file):
    with open(dlg_file, 'r') as file:
        lines = file.readlines()

    #
    efb_lines = [line.strip().split() for line in lines if 'DOCKED: USER    Estimated Free Energy of Binding' in line]

    if not efb_lines:
        return None

    #
    efb_values = [float(line[-3]) for line in efb_lines]

    #
    best_efb = min(efb_values)  #
    return best_efb

def process_dlg_files(dlg_pattern):
    dlg_files = glob.glob(dlg_pattern)

    if not dlg_files:
        print("no dlg file")
        return

    for dlg_file in dlg_files:
        best_efb = extract_efb_from_dlg(dlg_file)
        if best_efb is not None:
            print(f"{dlg_file} best Estimated Free Energy of Binding: {best_efb}")
        else:
            print(f"{dlg_file} no Estimated Free Energy of Binding")

#
process_dlg_files("*.dlg")


compound11.dlg best Estimated Free Energy of Binding: -14.15
MI23.dlg best Estimated Free Energy of Binding: -11.87
compound42.dlg best Estimated Free Energy of Binding: -12.18
CI006659.dlg best Estimated Free Energy of Binding: -11.45
calpain_inhibitors_I.dlg best Estimated Free Energy of Binding: -9.72
CI006653.dlg best Estimated Free Energy of Binding: -11.17
CI006630.dlg best Estimated Free Energy of Binding: -12.27
CI006656.dlg best Estimated Free Energy of Binding: -11.9
UWAJ246.dlg best Estimated Free Energy of Binding: -11.84
MPI8.dlg best Estimated Free Energy of Binding: -13.33
CI006628.dlg best Estimated Free Energy of Binding: -13.2
P0C6X7.dlg best Estimated Free Energy of Binding: -10.05
MPI3.dlg best Estimated Free Energy of Binding: -11.9
CI006660.dlg best Estimated Free Energy of Binding: -11.09
CI006629.dlg best Estimated Free Energy of Binding: -13.74
CI006634.dlg best Estimated Free Energy of Binding: -13.01
CI006632.dlg best Estimated Free Energy of Binding: -12.87


download files

In [ ]:
!ls -l
!mkdir covalentdock
!cp *.dlg covalentdock/
!cp *.pdbqt covalentdock/
!cp *.xml covalentdock/
!cp *.sdf covalentdock/
!tar cvf covalentdock.tar covalentdock/

from google.colab import files
files.download('covalentdock.tar')

total 173088
-rwxr-xr-x 1 root root   592832 Dec 17  2021 adgpu-v1.5.3_linux_ocl_128wi
-rw-r--r-- 1 root root   293908 Dec  4 16:25 Boceprevir.dlg
-rw-r--r-- 1 root root     4210 Dec  4 16:24 Boceprevir.pdbqt
-rw-r--r-- 1 root root     7665 Dec  4 16:23 Boceprevir.sdf
-rw-r--r-- 1 root root    93764 Dec  4 16:25 Boceprevir.xml
-rw-r--r-- 1 root root      142 Dec  4 16:16 boron-silicon-atom_par.dat
-rw-r--r-- 1 root root   261805 Dec  4 16:25 calpain_inhibitors_I.dlg
-rw-r--r-- 1 root root     3189 Dec  4 16:24 calpain_inhibitors_I.pdbqt
-rw-r--r-- 1 root root     5943 Dec  4 16:23 calpain_inhibitors_I.sdf
-rw-r--r-- 1 root root    79047 Dec  4 16:25 calpain_inhibitors_I.xml
-rw-r--r-- 1 root root   286418 Dec  4 16:26 Calpain_Inhibitor_XII.dlg
-rw-r--r-- 1 root root     3935 Dec  4 16:24 Calpain_Inhibitor_XII.pdbqt
-rw-r--r-- 1 root root     6447 Dec  4 16:23 Calpain_Inhibitor_XII.sdf
-rw-r--r-- 1 root root    92815 Dec  4 16:26 Calpain_Inhibitor_XII.xml
-rw-r--r-- 1 root root   308026

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>